In [1]:
import tweepy
import os
import pandas as pd
import yfinance as yf
import openai
import big_baller_moves
import json
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup

In [2]:
#Put your Bearer Token in the parenthesis below
client1 = tweepy.Client(bearer_token=big_baller_moves.bossman_tingz["twitter_api_key"])

# Authenticate
auth = tweepy.OAuth1UserHandler(big_baller_moves.bossman_tingz["consumer_key"], big_baller_moves.bossman_tingz["consumer_secret"], big_baller_moves.bossman_tingz["access_token"], 
                                big_baller_moves.bossman_tingz["access_token_secret"])

# Create API object
api = tweepy.API(auth)

# openai key
openai.api_key = big_baller_moves.bossman_tingz["openai_api_key"]

In [3]:
#enter username to get the user ID
username = "Heisenberg_100k"

try:
    user = client1.get_user(username=username)
    user_id = user_id = user.data.id
    print("User ID for", username, ":", user_id)
except Exception as e:
    print("Error:", e)

User ID for Heisenberg_100k : 1750349614786691072


In [ ]:
# #get btc price (not working atm)

# # Choose Token and Time
# timestamp_open = pd.Timestamp('2024-05-02 05:09:00', tz='UTC')
# unix_timestamp = int(timestamp_open.timestamp())
# unix_timestamp_24 = unix_timestamp + (24*60*60)
# unix_timestamp_48 = unix_timestamp + (48*60*60)
# ckey = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp}&localization=false"
# url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"
# ckey24 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_24}&localization=false"
# ckey48 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_48}&localization=false"

# try:
#         # Convert timestamp to Unix timestamp (Kraken API requires timestamp in seconds)
#         unix_timestamp = int(timestamp.timestamp())

#         # Kraken API endpoint for OHLC data
#         url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"

# response = requests.get(url)
#         if response.status_code == 200:
#             data = response.json()
#             if 'result' in data and 'XBTUSD' in data['result'] and data['result']['XBTUSD']:
#                 # The response contains a list of lists, where each inner list represents OHLC data for a specific interval
#                 # We extract the closing price from the last list (most recent data point)
#                 bitcoin_price = float(data['result']['XBTUSD'][-1][4])
#                 return bitcoin_price
#             else:
#                 print("No data available for the specified timestamp")
#                 return None
#         else:
#             print(f"Failed to retrieve data from Kraken API. Status code: {response.status_code}")
#             return None
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         return None


In [4]:
#get all tweets
tweets = client1.get_users_tweets(
    id= user_id,
    max_results=100,  # Number of tweets to retrieve (adjust as needed)
    tweet_fields=['id', 'text', 'created_at', 'entities', 'attachments'],  # Fields you want to retrieve for each tweet
    media_fields=['preview_image_url', 'url'],
    exclude = ['retweets','replies'],
    expansions=['attachments.media_keys', 'author_id']
)

df = pd.DataFrame(tweets.data)

In [55]:
#extract urls for tweets with images
def get_display_url(entities):
    if isinstance(entities, dict) and "urls" in entities:
        urls = entities["urls"]
        display_urls = [url.get("display_url") for url in urls]
        return display_urls
    else:
        return 0

# Apply the function to make the display_urls a column in our dataframe
df["image_url"] = df["entities"].apply(get_display_url)

selected_columns = ['id', 'text', 'created_at', 'image_url']

# Creating a new DataFrame with selected columns
heisenberg_tweets = df[selected_columns].copy()

In [6]:
client = openai
df1 = df.copy().head(5)
pd.set_option('display.max_colwidth', None)

In [7]:
#heisenberg
system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [8]:
#translates images to option play 
def get_response_image(text) :
    if text == 0:
        return "No image available"
    if not isinstance(text, str):
        print(text)
        text = text[0]
    try:
        response = client.chat.completions.create(model="gpt-4o", messages=[
                            {
                              "role": "user",
                              "content": [
                                {"type": "text", "text": "What kind of stock purchase is this image describing? If it is an option play like a call or a put please specify."},
                                {
                                  "type": "image_url",
                                  "image_url": {
                                    "url": text,
                                  },
                                },
                              ],
                            }
                          ], max_tokens=300,)
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass

In [9]:
def dynamic_prompting(text, sys_prompt, user_prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": sys_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": user_prompt},
                        {"type": "text", "text": text}
                    ]
                }
            ],
            max_tokens=300,
        )
        
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass


In [47]:
#CHAT
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

def get_jpg_url(driver, link):
    max_attempts = 10
    initial_wait = 0.5
    max_wait = 10

    for attempt in range(max_attempts):
        url = None
        try:
            if isinstance(link, list):  # Extract first element if URL is a list
                for u in link:
                    if "twitter" in u:
                        url = u
                        break
            elif not link:
                print("No Link Provided")
                return None
            else:
                url = link

            if not url:
                return None

            url = f'https://{url}'
            print(f"Attempting to access URL: {url}")

            driver.get(url)
            # Wait for potential redirects and the page to stabilize
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'article')))
            time.sleep(1)  # Extended sleep to ensure the page is loaded

            images = driver.find_elements(By.TAG_NAME, "img")
            for img in images:
                img_src = img.get_attribute('src')
                if 'media' in img_src:
                    print("img_src: " + img_src)
                    return img_src

            print("No media images found on the page")
            return None

        except Exception as e:
            print(f"General error processing URL {url}: {e}")

        wait_time = initial_wait * (2 ** attempt) + random.uniform(0, 1)
        wait_time = min(wait_time, max_wait)
        print(f"Retrying in {wait_time:.2f} seconds...")
        time.sleep(wait_time)

    print("Maximum attempts reached, aborting")
    return None

driver = initialize_webdriver()

try:
    heisenberg_tweets.loc[:, 'jpg_url'] = heisenberg_tweets['image_url'].apply(lambda x: get_jpg_url(driver, x))

finally:
    driver.quit()

https://pic.twitter.com/JQA5OMHePm
img_src: https://pbs.twimg.com/media/GMrdM1YWgAAbNjb?format=jpg&name=small


In [58]:
def contains_twitter_pic(url_list):
    if not url_list:
        return False
    return any('pic.twitter' in url for url in url_list)


,id,text,created_at,image_url,jpg_url
2,1793694236430434395,$VIX Pressed another 5 contracts\n\nTotal 50 contracts @ 2.98 avg.\n\n89 days for it to do something.\n\nEveryone is calling for a boring dull summer where VIX just dies.\n\nI went play contrarian and see that the VIX will have one more jump in it to 16-18’s within next 2-3 months.… https://t.co/BGSDDAGHpT https://t.co/dslHjMOuNR,2024-05-23 17:23:25+00:00,"[x.com/i/web/status/1…, pic.twitter.com/dslHjMOuNR]",https://pbs.twimg.com/media/GOR7ZqSaYAAPNmJ?format=jpg&name=small
5,1793294582316151110,"$NVDA Open\n\nER play for today\n\nRisk $2,050 to make $450 https://t.co/kadHdk8ML5",2024-05-22 14:55:20+00:00,[pic.twitter.com/kadHdk8ML5],https://pbs.twimg.com/media/GOMP6tpboAAp_Re?format=jpg&name=small
6,1793286225878761645,$PZZA Open\n\nTrying to catch a falling knife here. https://t.co/kXdh92KP3p,2024-05-22 14:22:08+00:00,[pic.twitter.com/kXdh92KP3p],https://pbs.twimg.com/media/GOMIUWKaMAAoILy?format=jpg&name=small
7,1792927921704710336,$WBA Open\n\nAnother contrarian play.\n\n241 days to do something.\n\nA move back to $20 should get a bagger on these near leaps. https://t.co/x4LiW2CwpL,2024-05-21 14:38:22+00:00,[pic.twitter.com/x4LiW2CwpL],https://pbs.twimg.com/media/GOHCcHgW0AE_cDV?format=jpg&name=small
8,1792926733584211971,$PTON Open\n\nThis is what I call a “why not” play.\n\nBinary. Either this goes to $0-1 or it goes to $7-10.\n\nAll hinders on a takeover imo.\n\nGot 241 days to do something. https://t.co/eraAKf9z3s,2024-05-21 14:33:38+00:00,[pic.twitter.com/eraAKf9z3s],https://pbs.twimg.com/media/GOHBW5ZXgAENuEg?format=jpg&name=small
10,1792613374087541228,$VIX Pressed.\n\nAdded 15 more. \n\nTotal 45 contracts at 2.98 avg.\n\n92 days for the VIX to do something. https://t.co/69oBnqD7tv,2024-05-20 17:48:28+00:00,[pic.twitter.com/69oBnqD7tv],https://pbs.twimg.com/media/GOCkXLVaMAM-NqN?format=jpg&name=small
11,1791524527786332521,$VIX DCA’ed and pressed\n\nMy only position in the port for now.\n\n30 contracts at 3.03 average. It’s down $200 bucks no biggie.\n\nI’m looking for VIX to steady here next few weeks. Could enter 11 handle briefly. Then head to 15-16. Should get a decent return on these. Have 95 days. https://t.co/t2qfxLCjDy,2024-05-17 17:41:46+00:00,[pic.twitter.com/t2qfxLCjDy],https://pbs.twimg.com/media/GNzGD_vbsAAUgi7?format=jpg&name=small
14,1791190695249838362,"$VIX Open\n\nRisk $4,650.\n\nHave 3+ months on these VIX calls.\n\nJust looking for 25-50% on these. Could take some time. https://t.co/9qf85HGxq0",2024-05-16 19:35:15+00:00,[pic.twitter.com/9qf85HGxq0],https://pbs.twimg.com/media/GNuWcVSboAAo90c?format=jpg&name=small
15,1791189980225863994,"$CABA Closed\n\nIn 16.05 (4-19-24)\n\nOut 12.40\n\n-23%\n-$1,095 https://t.co/ZyqTD8by6K",2024-05-16 19:32:24+00:00,[pic.twitter.com/ZyqTD8by6K],https://pbs.twimg.com/media/GNuVyzSbsAApokt?format=jpg&name=small
16,1791189613383766523,$PLTR Closed\n\nIn 22.04 (5-7-24)\n\nOut 21.78\n\n-1.2%\n-$104 https://t.co/JHUGNI69Hk,2024-05-16 19:30:57+00:00,[pic.twitter.com/JHUGNI69Hk],https://pbs.twimg.com/media/GNuVdZ_aAAAF9dp?format=jpg&name=small


In [ ]:
heisenberg_tweets.loc[:,'image_response'] = heisenberg_tweets['jpg_url'].apply(lambda x: get_response_image(x) if x != None else None)
heisenberg_tweets['image_response'] = heisenberg_tweets['image_response'].apply(lambda x: x.replace('$', '\$') if x and pd.notna(x) else x)
heisenberg_tweets['text'] = heisenberg_tweets['text'].apply(lambda x: x.replace('$', '\$') if pd.notna(x) else x)
heisenberg_tweets['full_response'] = heisenberg_tweets['text'].astype(str) + '  TRANSCRIBED IMAGE DATA: ' + heisenberg_tweets['image_response'].astype(str)

cwd = os.getcwd()
file_path = os.path.join(cwd, 'ht_unprocessed.xlsx')
heisenberg_tweets['created_at'] = heisenberg_tweets['created_at'].dt.tz_localize(None)
heisenberg_tweets = heisenberg_tweets.map(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
print(file_path)
heisenberg_tweets.to_excel(file_path, index=False)

In [13]:
heisenberg_tweets.head(10)

,id,text,created_at,image_url,jpg_url
0,1795170192571384017,"$IWM Watchlist.\n\nIf inflation PCE #’s come in light, small caps should rip.\n\nProbably will keep my VIX play at bay if so. But I guess we can be a short term bull and a mid term mini bear.",2024-05-27 19:08:21+00:00,0,None
1,1795168704562385037,Earnings play this week:\n\n$ULTA\n\nGoing long in this one tomorrow to end of week. Hoping for a pre er run up.,2024-05-27 19:02:26+00:00,0,None
2,1793694236430434395,$VIX Pressed another 5 contracts\n\nTotal 50 contracts @ 2.98 avg.\n\n89 days for it to do something.\n\nEveryone is calling for a boring dull summer where VIX just dies.\n\nI went play contrarian and see that the VIX will have one more jump in it to 16-18’s within next 2-3 months.… https://t.co/BGSDDAGHpT https://t.co/dslHjMOuNR,2024-05-23 17:23:25+00:00,"[x.com/i/web/status/1…, pic.twitter.com/dslHjMOuNR]",https://pbs.twimg.com/media/GOR7ZqSaYAAPNmJ?format=jpg&name=small
3,1793378137189065025,Ftw!,2024-05-22 20:27:21+00:00,0,None
4,1793341078600577524,"A part of me really really feels the $VIX is due for a nice little pop any time now.\n\nThat does not mean the $SPX comes crashing, no.\n\nCan easily see a 20-30% VIX pop and only a SPX 1-1.5% drop.",2024-05-22 18:00:06+00:00,0,None
...,...,...,...,...,...
93,1787197245634609268,Watchlist for this week (mostly earnings plays):\n\nLongs:\n\n$UBER\n$RACE\n$HOOD\n$GLD\n$XLE\n\nNeutrals with slight upward bias:\n\n$DIS\n$PLTR\n$OXY,2024-05-05 19:06:42+00:00,0,None
94,1786788566175232365,Kinda sucks. I was staring at AMD a couple days ago at 141-142. And it just ran off without me to 151.\n\nShucks.,2024-05-04 16:02:45+00:00,0,None
95,1786765311678750930,"Account Holdings and Account Value Update:\n\nCABA - bagholding. Had a few chances to cash out positive, but that's ok, small holdings.\n\nDKNG - continues to be annoying and stubborn. Had a few chances to cash out as well at even. Will just hodl.\n\nThis week I'm looking to buy some… https://t.co/lRsKVjgfH8 https://t.co/3wWXz3Kczo",2024-05-04 14:30:21+00:00,"[x.com/i/web/status/1…, pic.twitter.com/3wWXz3Kczo]",https://pbs.twimg.com/media/GMvdM11aYAAGNEQ?format=png&name=small
96,1786490904661913937,"Biggest Winners this week:\n\n$SPY call credit spread +$700\n$PYPL put credit spread +$480 (earnings play)\n$AMZN put credit spread +$410 (earnings play)\n\nAccount balance: $120,025.\n\nNext week I'm looking for $SPY max upside level at 515 with a potential to fill that 505 gap later on… https://t.co/w7zX8A2U71",2024-05-03 20:19:57+00:00,[x.com/i/web/status/1…],None


In [ ]:
#I've separated the prompting into two separate columns, one to get the ticker (here) and another to get the direction (below)

cols = ['id', 'created_at', 'full_response']
ht_dynamic = heisenberg_tweets[cols]
sys_prompt = """You are parsing tweets to interpret and synthesize information about stock plays. Reference examples as a guide to understand the format of the output.
Example:
Text: $PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. 
https://t.co/GpCKwDrfky 
TRANSCRIBED IMAGE DATA: This image describes a stock sale transaction, not an option play like a call or a put. 
Specifically, it details the sale of 500 shares of PARA (which is the ticker symbol for a stock) at an average fill price of $13.2401. 
It's a limit order set to sell at $13.24. The negative quantity (-500) indicates that shares are being sold rather than purchased.
Correct Output: [Close PARA Long]

Text: If $META closes above 450, I will do 1,000 jumping jacks.\n\n🙏
TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: \\$GOOG Closed\n\nIn .32 credit (Apr 30th)\n\nOut .05 debit\n\n+\\$135 profit\n\nSmall. Got dangerously close. But never worried. TRANSCRIBED IMAGE DATA: The image describes a vertical options spread for Google (GOOG) stock, specifically a put spread. Here are the details:\n\n- **Type of Spread**: Vertical Put Spread\n- **Underlying Asset**: GOOG (Google)\n- **Expiry Date**: May 3, 2024\n- **Strike Prices**: 165 (bought) and 162.5 (sold)\n\nDetails of the trade:\n- **Quantity**: 5 contracts\n- **Total Cost**: Since you are paying 0.05 per contract for 5 contracts, the total debit is 5 * 0.05 = \\$25.\n\nIn this trade, you are buying 5 put options at a strike price of 165 and selling 5 put options at a strike price of 162.5 for the same expiration date of May 3, 2024. This creates a vertical spread (specifically a bear put spread), aiming to profit from a decrease in the price of the underlying stock, Google, down to or below the lower strike price of 162.5.
Correct Output: [Close GOOG Short]

Text: \\$AMD Eyeing this name.\n\nIf it can get to 137-138ish tomorrow or next week, then I would buy there. \n\nWhy? 200dma magnet could act as major support.  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: \\$DIS Open\n\nEarnings play\n\nRisk \\$1,670 to make \\$330.\n\nBmo, implied about a 7pt move. https://t.co/nyMZGS23OZ  TRANSCRIBED IMAGE DATA: The image describes a vertical put spread (also known as a bull put spread). This is a type of options strategy that involves selling one put option and buying another put option at a lower strike price but with the same expiration date.\n\nHere's the breakdown of the trade based on the image:\n\n- This is for the stock with the ticker symbol "DIS" (Walt Disney Company), with options expiring on May 10, 2024.\n- The vertical spread involves the 108 and 106 strike prices, meaning you are dealing with 108/106 put options.\n- The strategy specified is a put vertical spread:\n  - Selling 10 put options at the 108 strike price.\n  - Buying 10 put options at the 106 strike price.\n- The prices for the transactions are:\n  - Sold (shorted) the 108 strike put options at \\$0.86 each.\n  - Bought (long) the 106 strike put options at \\$0.53 each.\n- The net credit received for the spread is \\$0.33 per share (since options typically represent 100 shares, the total net credit is \\$33 per contract).\n\nIn summary, the strategy is a bull put spread where you hope the price of Disney (DIS) stays above the higher strike price (108) by the expiration date so that both options expire worthless, and you keep the credit received upfront.
Correct Output: [Open DIS Long]

Text: Watchlist for next week:\n\nLong: \\$VIX\n\nShort: \\$FXI, \\$SPY\n\nNeutral: \\$NVDA\n\nSpeculative bounce play on watch: \\$SHOP  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: Two Biggest Losers this week:\n\nSPY call credit spread: -\\$5,130\n\nDIS put credit spread: -\\$1,670\n\nOuch.\n\nAccount Balance: \\$114,888.39.\n\n-\\$5,097.20 week over week.\n\nNot happy, but we look forward to next week. https://t.co/X67fSZOUY6  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: \\\\$FXI Open\\n\\nJust some yolo puts here looking for a healthy small pullback.\\n\\nIndex up 10 of last 11 days. https://t.co/PWuTtGhm5t  TRANSCRIBED IMAGE DATA: The image describes the purchase of a put option. Specifically, the details are as follows:\\n\\n- Ticker Symbol: FXI 100 (This commonly refers to an exchange-traded fund (ETF) that tracks the performance of the top 100 Chinese companies).\\n- Weekly options expiring on: May 10th, 2024.\\n- Strike Price: 27.5.\\n- Type: Put option (designated by the "P").\\n- Quantity: 15 contracts.\\n- Price: \\\\$0.44 per contract (with a limit order as indicated by "LMT").\\n- Trade fill date and time: May 6th, 2024, at 6:32 AM.\\n\\nThis indicates that the trader bought 15 put options at a strike price of 27.5, and they paid \\\\$0.44 per option contract.
Correct Output: [Open FXI Short]

Text: Hang Seng going bonkers overnight.\\nIf it sticks by US open, these FXI calls should be close to 100%!  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: \\\\$GME selling call spreads on watch.  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: For those who want to play the VIX but cannot because cough Robinhood cough, then I suggest SPY puts. It's pretty much the next best alternative.\\n\\nDon't give me that UVXY crap imo.  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]
"""
user_prompt = "Is this tweet referring to the opening or closing of a stock position? If it is, please also list the corresponding ticker and whether it is long or short. If it is not referring to the opening or closing of a position, simply put neither. Please respond in the possible formats: [Open/Close TICKER Long/Short] or [Neither]. If the tweet refers to multiple positions, list them all in a comma separated list."
ht_dynamic['result'] = ht_dynamic['full_response'].apply(
    lambda text: dynamic_prompting(text, sys_prompt, user_prompt)
)


In [ ]:
cwd = os.getcwd()
file_path = os.path.join(cwd, 'tweet_consolidated.xlsx')
ht_dynamic['created_at'] = ht_dynamic['created_at'].dt.tz_localize(None)
ht_dynamic = ht_dynamic.map(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
print(file_path)
ht_dynamic.to_excel(file_path, index=False)

In [ ]:
#old

#ht_df.loc[:,'image_response'] = ht_df['jpg_url'].apply(lambda x: get_response_image(x) if x != None else None)
#ht_df['image_response'] = ht_df['image_response'].apply(lambda x: x.replace('$', '\$') if x and pd.notna(x) else x)
#ht_df['text'] = ht_df['text'].apply(lambda x: x.replace('$', '\$') if pd.notna(x) else x)
#ht_df['full_response'] = ht_df['text'].astype(str) + '  TRANSCRIBED IMAGE DATA: ' + ht_df['image_response'].astype(str)
#ht_df['buy_or_sell'] = ht_df['full_response'].apply(buy_or_sell_GPT_k_shot)
#ht_df['buy_or_sell_2'] = ht_df['full_response'].apply(buy_or_sell_GPT_k_shot)
#ht_df['buy_or_sell_3'] = ht_df['full_response'].apply(buy_or_sell_GPT_k_shot)
#ht_df['all equal'] =  (ht_df['buy_or_sell'] == ht_df['buy_or_sell_2']) & (ht_df['buy_or_sell_2'] == ht_df['buy_or_sell_3'])
#ht_df['full_response'] = ht_df['text'].astype(str) + '  TRANSCRIBED IMAGE DATA: ' + ht_df['image_response'].astype(str)
#ht_df['buy_or_sell'] = ht_df['full_response'].apply(buy_or_sell_GPT)
#ht_df['buy_or_sell_2'] = ht_df['full_response'].apply(buy_or_sell_GPT)
#ht_df['buy_or_sell_3'] = ht_df['full_response'].apply(buy_or_sell_GPT)
#ht_df['all equal'] =  (ht_df['buy_or_sell'] == ht_df['buy_or_sell_2']) & (ht_df['buy_or_sell_2'] == ht_df['buy_or_sell_3'])
#ht_df['2-3 equal'] =   (ht_df['buy_or_sell_2'] == ht_df['buy_or_sell_3'])
#ht_df['buy_or_sell_4'] = ht_df['full_response'].apply(buy_or_sell_GPT_k_shot)
#ht_df['buy_or_sell_5'] = ht_df['full_response'].apply(buy_or_sell_GPT_k_shot)
#ht_df['buy_or_sell_6'] = ht_df['full_response'].apply(buy_or_sell_GPT_k_shot)
#ht_df['rnd 2 all equal'] =  (ht_df['buy_or_sell_4'] == ht_df['buy_or_sell_5']) & (ht_df['buy_or_sell_5'] == ht_df['buy_or_sell_6'])
#selected_cols = ['created_at', 'full_response', 'buy_or_sell','buy_or_sell_2','buy_or_sell_3','all equal',
#                 '2-3 equal','buy_or_sell_4','buy_or_sell_5','buy_or_sell_6','rnd 2 all equal']
#ht_fin = ht_df[selected_cols]
#cwd = os.getcwd()
#file_path = os.path.join(cwd, 'tweet_buy_or_sell_3.xlsx')
#ht_fin['created_at'] = ht_fin['created_at'].dt.tz_localize(None)
#ht_fin = ht_fin.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
#print(file_path)
#ht_fin.to_excel(file_path, index=False)